# PassGAN

This Jupyter notebook is mainly for debugging, has same functionality to "sample.py".

## Dependencies

In [1]:
# Libraries
import os
import time
import pickle
import argparse
import tensorflow as tf
import numpy as np

# Files
import tflib as lib
import tflib.ops.l2
import tflib.ops.con1
import utils
import mod1


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# TensorFlow virsion
print(tf.__version__)

2.16.1


## Parameters

To simplify migration process between Jupyter Notebook and .py format, we created virtual ArgumentParser class.

In [23]:
# Class to virtualize ArgumentParser
class VirtualArgparse:
    
    # Name of directory where pickle dictyonary is stored
    input_dir = "pretrained"
    
    # TensorFlow checkpoint
    checkpoint = "pretrained/checkpoints/checkpoint_200000.ckpt"
    
    # Name of file to output generated passwords
    output = "generated_pass1.txt"
    
    # Batch size
    batch_size = 1024
    
    # Number of password to generate
    num_samples = 1000000
    
    # Max length of password to generate (Training-Dependent)
    seq_length = 10
    
    # Number of nodes on hidden layers (Training-Dependent)
    layer_dim = 128

In [24]:
# Virtualize ArgumentParser instance
args = VirtualArgparse()

## Importing Dictionary

In [25]:
# Dictionary
with open(os.path.join(args.input_dir, 'charmap.pickle'), 'rb') as f:
    charmap = pickle.load(f, encoding='latin1')

# Reverse-Dictionary
with open(os.path.join(args.input_dir, 'charmap_inv.pickle'), 'rb') as f:
    inv_charmap = pickle.load(f, encoding='latin1')

## Creating Generator

In [28]:
fake_inputs = mod1.Generator(args.batch_size, args.seq_length, args.layer_dim, len(charmap))

Linear layer: Generator.Input, Input dim: 128, Output dim: 1280
Weight values shape: (128, 1280), dtype: float32
Created weight: <tf.Variable 'Generator.Input/Generator.Input.W:0' shape=(128, 1280) dtype=float32_ref>
Result shape: (1024, 1280), dtype: <dtype: 'float32'>


## Starting TensorFlow Session

In [35]:


def generate_samples(session, fake_inputs, inv_charmap):
    samples = session.run(fake_inputs)
    print(f"Shape of generated samples before argmax: {samples.shape}")
    samples = np.argmax(samples, axis=2)
    print(f"Shape of generated samples after argmax: {samples.shape}")
    decoded_samples = []
    for i in range(len(samples)):
        decoded = []
        for j in range(len(samples[i])):
            decoded.append(inv_charmap[samples[i][j]])
        decoded_samples.append(tuple(decoded))
    return decoded_samples

def save(samples, output):
    with open(output, 'a') as f:
        for s in samples:
            s = "".join(s).replace('`', '')
            f.write(s + "\n")

# Assuming args, fake_inputs, inv_charmap are already defined
with tf.compat.v1.Session() as session:
    saver = tf.compat.v1.train.Saver()

    # Restore model
    saver.restore(session, args.checkpoint)

    samples = []
    then = time.time()
    start = time.time()
    for i in range(int(args.num_samples / args.batch_size)):
        samples.extend(generate_samples(session, fake_inputs, inv_charmap))

        # Append to output file every 1000 batches
        if i % 1000 == 0 and i > 0:
            save(samples, args.output)
            samples = []  # flush
            print(f"Wrote {1000 * args.batch_size} samples to {args.output} in {time.time() - then:.2f} seconds. {i * args.batch_size} total.")
            then = time.time()

    save(samples, args.output)

print(f"\nFinished in {time.time() - start:.2f} seconds")


INFO:tensorflow:Restoring parameters from pretrained/checkpoints/checkpoint_200000.ckpt


InvalidArgumentError: Graph execution error:

Detected at node 'Reshape_19' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\aswan\project\project\env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    File "c:\Users\aswan\project\project\env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    File "c:\Users\aswan\project\project\env\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    File "c:\Users\aswan\project\project\env\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start
    File "C:\Program Files\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
    File "C:\Program Files\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once
    File "C:\Program Files\Python311\Lib\asyncio\events.py", line 80, in _run
    File "c:\Users\aswan\project\project\env\Lib\site-packages\ipy

In [20]:
# with tf.compat.v1.Session() as session:

#     def generate_samples():
#         samples = session.run(fake_inputs)
#         samples = np.argmax(samples, axis=2)
#         decoded_samples = []
#         for i in range(len(samples)):
#             decoded = []
#             for j in range(len(samples[i])):
#                 decoded.append(inv_charmap[samples[i][j]])
#             decoded_samples.append(tuple(decoded))
#         return decoded_samples

#     def save(samples):
#         with open(args.output, 'a') as f:
#                 for s in samples:
#                     s = "".join(s).replace('`', '')
#                     f.write(s + "\n")

#     saver = tf.compat.v1.train.Saver()
#     saver.restore(session, args.checkpoint)

#     samples = []
#     then = time.time()
#     start = time.time()
#     for i in range(int(args.num_samples / args.batch_size)):
        
#         samples.extend(generate_samples())

#         # append to output file every 1000 batches
#         if i % 1000 == 0 and i > 0: 
            
#             save(samples)
#             samples = [] # flush

#             print('wrote {} samples to {} in {:.2f} seconds. {} total.'.format(1000 * args.batch_size, args.output, time.time() - then, i * args.batch_size))
#             then = time.time()
    
#     save(samples)
# print('\nFinished in {:.2f} seconds'.format(time.time() - start))